In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samaokasha","key":"6777a41d034bf7c33e4fac54cefd8688"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download samaokasha/ais477-lab

Dataset URL: https://www.kaggle.com/datasets/samaokasha/ais477-lab
License(s): unknown
 80% 34.0M/42.3M [00:00<00:00, 45.8MB/s]
100% 42.3M/42.3M [00:00<00:00, 60.1MB/s]


In [ ]:
!unzip ais477-lab.zip

Archive:  ais477-lab.zip
  inflating: output.csv              


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 18.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=72ee89b1cd88e0c73dd3c2efb8756bed26843a61050870e17e7cdbc232015406
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from sklearn.ensemble import ExtraTreesClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from pyspark.ml.feature import StringIndexer
# from pyspark.ml.classification import ExtraTreesClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.feature import ChiSqSelector
# from pyspark.ml.feature import VectorAssembler, StringIndexer, VarianceThresholdSelector
from pyspark.ml.feature import VarianceThresholdSelector
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Feature_Selection_and_Training") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


In [ ]:
# Step 2: Load the dataset using PySpark
dataset_path = "/content/output.csv"
df = spark.read.csv(dataset_path, header=True, inferSchema=True)

In [ ]:
from pyspark.sql.functions import col
# Filter out string columns
numeric_cols = [col_name for col_name, col_type in df.dtypes if col_type != 'string']

# Replace null values in numeric columns with a default value (e.g., 0) using a single call
fill_dict = {col_name: 0 for col_name in numeric_cols}
df_filled = df.na.fill(fill_dict)

# Cache the DataFrame to avoid recomputation
df_filled.cache()

DataFrame[_c0: int, 0: string, 1: int, 2: int, 3: int, 4: int, 5: int, 6: int, 7: int, 8: int, 9: int, 10: int, 11: int, 12: int, 13: int, 14: int, 15: int, 16: int, 17: int, 18: int, 19: int, 20: int, 21: int, 22: int, 23: int, 24: int, 25: int, 26: int, 27: int, 28: int, 29: int, 30: int, 31: int, 32: int, 33: int, 34: int, 35: int, 36: int, 37: int, 38: int, 39: int, 40: int, 41: int, 42: int, 43: int, 44: int, 45: int, 46: int, 47: int, 48: int, 49: int, 50: int, 51: int, 52: int, 53: int, 54: int, 55: int, 56: int, 57: int, 58: int, 59: int, 60: int, 61: int, 62: int, 63: int, 64: int, 65: int, 66: int, 67: int, 68: int, 69: int, 70: int, 71: int, 72: int, 73: int, 74: int, 75: int, 76: int, 77: int, 78: int, 79: int, 80: int, 81: int, 82: int, 83: int, 84: int, 85: int, 86: int, 87: int, 88: int, 89: int, 90: int, 91: int, 92: int, 93: int, 94: int, 95: int, 96: int, 97: int, 98: int, 99: int, 100: int, 101: int, 102: int, 103: int, 104: int, 105: int, 106: int, 107: int, 108: in

In [ ]:
# Assemble features into a single vector
vectorAssembler = VectorAssembler(inputCols=numeric_cols[:-1], outputCol="features")
assembled_df = vectorAssembler.transform(df_filled)

In [ ]:
# Automatically index features and handle continuous/categorical features
feature_indexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=10)
indexed_df = feature_indexer.fit(assembled_df).transform(assembled_df)

In [ ]:
# Convert the label column to categorical values
label_indexer = StringIndexer(inputCol="Label", outputCol="indexedLabel")
indexed_df = label_indexer.fit(indexed_df).transform(indexed_df)

In [ ]:
# Ensure indexedLabel is of integer type
indexed_df = indexed_df.withColumn("indexedLabel", col("indexedLabel").cast("integer"))

In [ ]:
# Define the Variance Threshold Selector
selector = VarianceThresholdSelector(featuresCol="features", outputCol="selectedFeatures")

# Set the threshold value
# threshold_value = 5.500000000000001e-06
threshold_value = 0.0000027500000000000005
selector.setVarianceThreshold(threshold_value)


VarianceThresholdSelector_4c9719d28e5e

In [ ]:
# Fit the selector to the data
selector_model = selector.fit(indexed_df)

In [ ]:
# Transform the data to select features
selected_df = selector_model.transform(indexed_df)

In [ ]:
# Extract the selected feature indices
selected_features = selector_model.selectedFeatures

In [ ]:
# Show the number of selected features
num_selected_features = len(selector_model.selectedFeatures)
print(f"Number of selected features: {num_selected_features}")

Number of selected features: 4727


In [ ]:
# Save the selected DataFrame to disk
selected_df.write.mode("overwrite").parquet("/content/selected_df.parquet")


In [ ]:
from google.colab import files

# Compress the directory into a zip file
!zip -r /content/selected_df.zip /content/selected_df.parquet

# Download the zip file
files.download('/content/selected_df.zip')


  adding: content/selected_df.parquet/ (stored 0%)
  adding: content/selected_df.parquet/.part-00043-a5406211-8559-4291-bfa9-1c496091ceca-c000.snappy.parquet.crc (deflated 18%)
  adding: content/selected_df.parquet/part-00047-a5406211-8559-4291-bfa9-1c496091ceca-c000.snappy.parquet (deflated 77%)
  adding: content/selected_df.parquet/part-00044-a5406211-8559-4291-bfa9-1c496091ceca-c000.snappy.parquet (deflated 77%)
  adding: content/selected_df.parquet/.part-00046-a5406211-8559-4291-bfa9-1c496091ceca-c000.snappy.parquet.crc (deflated 18%)
  adding: content/selected_df.parquet/.part-00054-a5406211-8559-4291-bfa9-1c496091ceca-c000.snappy.parquet.crc (deflated 16%)
  adding: content/selected_df.parquet/.part-00034-a5406211-8559-4291-bfa9-1c496091ceca-c000.snappy.parquet.crc (deflated 20%)
  adding: content/selected_df.parquet/part-00009-a5406211-8559-4291-bfa9-1c496091ceca-c000.snappy.parquet (deflated 81%)
  adding: content/selected_df.parquet/part-00007-a5406211-8559-4291-bfa9-1c496091c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import zipfile
# Step 2: Unzip the file
with zipfile.ZipFile('selected_df.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
# Step 4: Read the Parquet file into a DataFrame
selected_df = spark.read.parquet('/content/content/selected_df.parquet')

In [ ]:
# Verify the DataFrame
selected_df.printSchema()
selected_df.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- 0: string (nullable = true)
 |-- 1: integer (nullable = true)
 |-- 2: integer (nullable = true)
 |-- 3: integer (nullable = true)
 |-- 4: integer (nullable = true)
 |-- 5: integer (nullable = true)
 |-- 6: integer (nullable = true)
 |-- 7: integer (nullable = true)
 |-- 8: integer (nullable = true)
 |-- 9: integer (nullable = true)
 |-- 10: integer (nullable = true)
 |-- 11: integer (nullable = true)
 |-- 12: integer (nullable = true)
 |-- 13: integer (nullable = true)
 |-- 14: integer (nullable = true)
 |-- 15: integer (nullable = true)
 |-- 16: integer (nullable = true)
 |-- 17: integer (nullable = true)
 |-- 18: integer (nullable = true)
 |-- 19: integer (nullable = true)
 |-- 20: integer (nullable = true)
 |-- 21: integer (nullable = true)
 |-- 22: integer (nullable = true)
 |-- 23: integer (nullable = true)
 |-- 24: integer (nullable = true)
 |-- 25: integer (nullable = true)
 |-- 26: integer (nullable = true)
 |-- 27: integer (nullabl

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Persist the selected DataFrame in memory/disk
selected_df.persist()

DataFrame[_c0: int, 0: string, 1: int, 2: int, 3: int, 4: int, 5: int, 6: int, 7: int, 8: int, 9: int, 10: int, 11: int, 12: int, 13: int, 14: int, 15: int, 16: int, 17: int, 18: int, 19: int, 20: int, 21: int, 22: int, 23: int, 24: int, 25: int, 26: int, 27: int, 28: int, 29: int, 30: int, 31: int, 32: int, 33: int, 34: int, 35: int, 36: int, 37: int, 38: int, 39: int, 40: int, 41: int, 42: int, 43: int, 44: int, 45: int, 46: int, 47: int, 48: int, 49: int, 50: int, 51: int, 52: int, 53: int, 54: int, 55: int, 56: int, 57: int, 58: int, 59: int, 60: int, 61: int, 62: int, 63: int, 64: int, 65: int, 66: int, 67: int, 68: int, 69: int, 70: int, 71: int, 72: int, 73: int, 74: int, 75: int, 76: int, 77: int, 78: int, 79: int, 80: int, 81: int, 82: int, 83: int, 84: int, 85: int, 86: int, 87: int, 88: int, 89: int, 90: int, 91: int, 92: int, 93: int, 94: int, 95: int, 96: int, 97: int, 98: int, 99: int, 100: int, 101: int, 102: int, 103: int, 104: int, 105: int, 106: int, 107: int, 108: in

In [ ]:
import os
# Directory to save the chunks
os.makedirs("chunks", exist_ok=True)

In [ ]:
# Convert the selected features and labels to Pandas DataFrame in batches
batch_size = 10000
start = 0
chunk_index = 0
total_rows = 384705

while start < total_rows:
    end = start + batch_size
    batch_df = selected_df.select("selectedFeatures", "indexedLabel").limit(end).offset(start).toPandas()

    # Save each chunk to a file
    np.save(f'chunks/X_chunk_{chunk_index}.npy', np.array(batch_df["selectedFeatures"].tolist()))
    np.save(f'chunks/y_chunk_{chunk_index}.npy', np.array(batch_df["indexedLabel"].tolist()))

    start = end
    chunk_index += 1

print(f"Total chunks created: {chunk_index}")

In [4]:
import zipfile
# Step 2: Unzip the file
with zipfile.ZipFile('/content/chunks.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/chunks')

In [ ]:
import numpy as np
import os

# Load all chunks
chunk_files = [f for f in os.listdir("chunks") if f.startswith("X_chunk")]
X_chunks = [np.load(os.path.join("chunks", f)) for f in sorted(chunk_files)]

chunk_files = [f for f in os.listdir("chunks") if f.startswith("y_chunk")]
y_chunks = [np.load(os.path.join("chunks", f)) for f in sorted(chunk_files)]

# Concatenate all chunks
X = np.concatenate(X_chunks, axis=0)
y = np.concatenate(y_chunks, axis=0)

In [ ]:
import numpy as np
import glob

# Directory where the chunks are saved
chunks_dir = 'chunks/'

# Get list of all X and y chunk files
X_files = sorted(glob.glob(chunks_dir + 'X_chunk_*.npy'))
y_files = sorted(glob.glob(chunks_dir + 'y_chunk_*.npy'))

# Initialize empty lists to store the data
X_list = []
y_list = []

# Load and concatenate the chunks
for X_file, y_file in zip(X_files, y_files):
    X_list.append(np.load(X_file))
    y_list.append(np.load(y_file))

# Concatenate all chunks
X = np.concatenate(X_list, axis=0)
y = np.concatenate(y_list, axis=0)


In [9]:
# Save the concatenated arrays to disk
np.save('/content/X_concatenated2.npy', X)
np.save('/content/y_concatenated.npy', y)

In [7]:
from google.colab import files

# Download the concatenated arrays
files.download('/content/X_concatenated.npy')
# files.download('/content/y_concatenated.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Convert the selected features and labels to Pandas DataFrame in batches
batch_size = 10000
start = 0

X = []
y = []

total_rows = 384705

while start < total_rows:
    end = start + batch_size
    batch_df = selected_df.select("selectedFeatures", "indexedLabel").limit(end).offset(start).toPandas()
    X.extend(batch_df["selectedFeatures"].tolist())
    y.extend(batch_df["indexedLabel"].tolist())
    start = end

X = np.array(X)
y = np.array(y)

In [ ]:
np.save('/content/X.npy', X)
np.save('/content/y.npy', y)

In [ ]:
from google.colab import files

# Download X.npy
files.download('/content/X.npy')

# Download y.npy
files.download('/content/y.npy')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
def train_model(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with Adam optimizer and binary cross-entropy loss
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [ ]:
# Train the model
input_dim = X_train.shape[1]
model = train_model(input_dim)
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test),
                    callbacks=[ModelCheckpoint('best_model_weights.h5', monitor='val_accuracy', verbose=1,
                                               save_best_only=True, mode='max'),
                               EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
9615/9618 [============================>.] - ETA: 0s - loss: -4592.5981 - accuracy: 0.3188
Epoch 1: val_accuracy improved from -inf to 0.25396, saving model to best_model_weights.h5
9618/9618 [==============================] - 130s 13ms/step - loss: -4594.6289 - accuracy: 0.3187 - val_loss: -13310.5986 - val_accuracy: 0.2540
Epoch 2/10
   5/9618 [..............................] - ETA: 2:11 - loss: -16510.3965 - accuracy: 0.2875

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9614/9618 [============================>.] - ETA: 0s - loss: -27421.2559 - accuracy: 0.3093
Epoch 2: val_accuracy improved from 0.25396 to 0.25526, saving model to best_model_weights.h5
9618/9618 [==============================] - 118s 12ms/step - loss: -27419.9453 - accuracy: 0.3093 - val_loss: -51947.9141 - val_accuracy: 0.2553
Epoch 3/10
9617/9618 [============================>.] - ETA: 0s - loss: -68635.5938 - accuracy: 0.3020
Epoch 3: val_accuracy did not improve from 0.25526
9618/9618 [==============================] - 118s 12ms/step - loss: -68629.9062 - accuracy: 0.3020 - val_loss: -105356.9062 - val_accuracy: 0.2540
Epoch 4/10
9616/9618 [============================>.] - ETA: 0s - loss: -127774.7891 - accuracy: 0.3000
Epoch 4: val_accuracy did not improve from 0.25526
9618/9618 [==============================] - 118s 12ms/step - loss: -127781.2422 - accuracy: 0.3000 - val_loss: -249935.3438 - val_accuracy: 0.2542
Epoch 5/10
9618/9618 [==============================] - ETA: 0s 

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
# Print model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2,
                    callbacks=[ModelCheckpoint('model_weights.h5', save_best_only=True),
                               EarlyStopping(patience=10)])

In [ ]:
# Print the accuracy
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [ ]:
print(f"Training accuracy over epochs: {train_acc}")
print(f"Validation accuracy over epochs: {val_acc}")

In [ ]:
# Find the best epoch based on validation accuracy
best_epoch = np.argmax(val_acc) + 1
best_val_acc = val_acc[best_epoch - 1]

print(f"Best validation accuracy: {best_val_acc} at epoch {best_epoch}")

In [ ]:
# Download the model weights
files.download('model_weights.h5')

In [ ]:
# Download the model weights
files.download('/content/model_weights.h5')

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [11]:
# Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Model Accuracy: {accuracy}")

Random Forest Model Accuracy: 0.9532758867183946


In [12]:
# Save the model using pickle
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

In [ ]:
# Stop SparkSession
spark.stop()